In [1]:
# Before running move this jupyter notbook to root folder
import sqlite3
import pandas as pd
from core.article import Article
from core.load_database import loadDatabase
from core.bom import Bom
from core import SQL_DB_DIR

In [2]:
# Access to Bom db
db, a_db, art_db = loadDatabase()

>>  Loading database ....
>>  Database loaded successfully
>>  Looking up for Articles rate file...
>>  Successfully loaded "data/articles.csv".


In [3]:
# SQL Headers and Query
headers = [
    "father",
    "fathername",
    "brand",
    "child",
    "childname",
    "childqty",
    "childuom",
    "childrate",
    "childtype",
    "producttype",
    "mrp",
    "process",
    "processorder",
    "application",
]

query = """
with recursive cte ({0}) as (
    select     {0}
    from       bom
    where      father = "{1}"
    union all
    select     {2}
    from       bom p
    inner join cte
            on p.father = cte.child
    )
select * from cte
"""

# EXECUTION

In [4]:
con = sqlite3.connect(SQL_DB_DIR)

In [5]:
for i, row in art_db.iterrows():
    rates = tuple(row[1:])
    article = Article.from_bulk_list(row[0],rates)
    bom = Bom(article)
    bom.createFinalBom(db)
    df = bom.bom_df

In [6]:
parent_headers = ",".join(["p." + x for x in headers])
header = ",".join(headers)
for i, row in art_db.iterrows():
    rates = tuple(row[1:])
    article = Article.from_bulk_list(row[0],rates)
    q = query.format(header, article.get_mc_name.upper(),parent_headers )
    df = pd.read_sql_query(q, con)

In [7]:
import timeit

def sql_process():
    for i, row in art_db.iterrows():
        rates = tuple(row[1:])
        article = Article.from_bulk_list(row[0],rates)
        q = query.format(header, article.get_mc_name.upper(),parent_headers )
        df = pd.read_sql_query(q, con)

def pandas_process():
    for i, row in art_db.iterrows():
        rates = tuple(row[1:])
        article = Article.from_bulk_list(row[0],rates)
        bom = Bom(article)
        bom.createFinalBom(db)
        df = bom.bom_df

In [8]:
sql_timing = timeit.timeit(sql_process, number=10)
pandas_timing = timeit.timeit(pandas_process, number=10)
print(f"SQL bom fetching took {sql_timing} seconds")
print(f"Pandas bom fetching took {pandas_timing} seconds")

SQL bom fetching took 4.1931218 seconds
Pandas bom fetching took 1.4916061000000127 seconds
